# Al surface - Conventional cell


In this notebook we will do abinit calculation for a Aluminium surface.
For simplicity we will use convetional cell for the calculations.

Basic overview:
- Bulk - structure optimisation:
  - total energy
  - lattice parameters
- Surfaces:
  - the relaxation of surface atoms?
  - surface energy


In [ ]:
%matplotlib inline

import warnings 
import numpy as np
import matplotlib.pylab as plt
import plotly.graph_objects as go

from itertools import product

from abipy import abilab, flowtk

from pymatgen.core import Structure, Lattice
from pseudo_dojo import OfficialDojoTable

from jupyter_jsmol import JsmolView
from jupyter_jsmol.pymatgen import quick_view

from pymatgen_plotly import Figure

abilab.enable_notebook() # This line tells AbiPy we are running inside a notebook
warnings.filterwarnings("ignore")  # Ignore warnings


pseudos_table = OfficialDojoTable.from_dojodir("ONCVPSP-PBE-PDv0.4", accuracy='standard')


# Bulk

## Initial structure

In [ ]:
lattice_constant = 4.041265916093099 # angstrom
lattice_constant

In [ ]:
def fcc_conv(element: str, a: float):
    # lattice=Lattice.cubic(a)
    structure = Structure(
        lattice=Lattice([[a, 0, 0], [0, a, 0], [0, 0, a]]),
        species=4 * [element],
        coords=[[0, 0, 0], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]],
    )
    return structure

struct_initial = fcc_conv('Al', lattice_constant)
struct_initial

In [ ]:
quick_view(struct_initial)



In [ ]:
fig = Figure()
fig.add_structure(struct_initial)
fig.add_unitcell(struct_initial)
fig.show()

# Single SCF calculation

In [ ]:
def build_inp_scf(structure, ecut=20, tsmear=0.01, shifted=False):
        """Generate input for optimization of the lattice parameter
        at fixed number of k points and broadening.
        """

        inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)

        inp["chkprim"] = 0
        # inp['chksymbreak'] = None

        inp["ecut"] = ecut
        inp["tsmear"] = tsmear

        # Definition of the k-point grid
        inp["kptopt"] = 1
        inp["ngkpt"] = [8, 8, 8]
        
        inp["nshiftk"] = 1
        if shifted:
            inp["shiftk"] = [0.5, 0.5, 0.5] 
        else:
            inp["shiftk"] = [0.0, 0.0, 0.0]

        inp["occopt"] = 6
        # inp["nband"] = 3 * len(structure) + 6 # NOTE: This value is specific for Al

        ## Definition of the SCF procedure
        # Maximal number of SCF cycles
        inp["nstep"] = 50

        # Will stop when, twice in a row, the difference
        # between two consecutive evaluations of total energy
        # differ by less than toldfe (in Hartree)
        # This value is way too large for most realistic studies of materials
        # inp['toldfe'] = 1e-8
        inp["tolvrs"] = 1e-10

        return inp

inp_scf = build_inp_scf(struct_initial)
# inp_scf

In [ ]:
def build_flow_scf(inp, workdir, force=False):

    flow = flowtk.Flow.from_inputs(workdir, inputs=inp)

    if force:
        flow.rmtree()

    return flow

flow_scf = build_flow_scf(inp_scf, "conv_cell/00_scf", force=True)

try:
    scheduler = flow_scf.make_scheduler()
    scheduler.start()
except:
    pass


In [ ]:
out_scf = abilab.abiopen('conv_cell/00_scf/w0/t0/outdata/out_GSR.nc')
out_scf.energy

## Relaxation

In [ ]:
def build_inp_relax(structure, ecut=20, tsmear=0.01, shifted=False):
        """Generate input for optimization of the lattice parameter
        at fixed number of k points and broadening.
        """

        inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)

        inp["chkprim"] = 0
        # inp['chksymbreak'] = None

        inp["ecut"] = ecut
        inp["tsmear"] = tsmear

        # Definition of the k-point grid
        inp["kptopt"] = 1
        inp["ngkpt"] = [8, 8, 8]
        
        inp["nshiftk"] = 1
        if shifted:
            inp["shiftk"] = [0.5, 0.5, 0.5] 
        else:
            inp["shiftk"] = [0.0, 0.0, 0.0]

        inp["occopt"] = 6
        inp["nband"] = 3 * len(structure) + 6 # NOTE: This value is specific for Al

        ## Optimization of the lattice parameters
        inp["optcell"] = 1
        inp["ionmov"] = 2

        inp["ntime"] = 10
        inp["dilatmx"] = 1.05
        inp["ecutsm"] = 0.5

        ## Definition of the SCF procedure
        # Maximal number of SCF cycles
        inp["nstep"] = 50

        # Will stop when, twice in a row, the difference
        # between two consecutive evaluations of total energy
        # differ by less than toldfe (in Hartree)
        # This value is way too large for most realistic studies of materials
        # inp['toldfe'] = 1e-8
        inp["tolvrs"] = 1e-10

        return inp

inp_relax=build_inp_relax(struct_initial)
# inp_relax

In [ ]:
def build_flow_relax(inp, workdir, force=False):

    flow = flowtk.Flow.from_inputs(workdir, inputs=inp)

    if force:
        flow.rmtree()

    return flow

flow_relax = build_flow_relax(inp_relax, "conv_cell/01_relax")

try:
    scheduler = flow_relax.make_scheduler()
    scheduler.start()
except:
    pass


In [ ]:
abo = abilab.abiopen("conv_cell/01_relax/w0/t0/run.abo")

struct_relaxed = abo.final_structure
struct_relaxed

In [ ]:
lattice_constant_relaxed = struct_relaxed.lattice.a

In [ ]:
def build_inp_dftp(structure, ecut=20, tsmear=0.01, shifted=False):

    inp = abilab.AbinitInput(structure=structure, pseudos=pseudos_table)

    inp["chkprim"] = 0
    # inp['chksymbreak'] = None

    inp["ecut"] = ecut
    inp["tsmear"] = tsmear

    # Definition of the k-point grid
    inp["kptopt"] = 1
    inp["ngkpt"] = [8, 8, 8]

    inp["nshiftk"] = 1
    if shifted:
        inp["shiftk"] = [0.5, 0.5, 0.5] 
    else:
        inp["shiftk"] = [0.0, 0.0, 0.0]

    inp["occopt"] = 6
    inp["nband"] = 3 * len(structure) + 6 # NOTE: This value is specific for Al


    ## Definition of the SCF procedure
    # Maximal number of SCF cycles
    inp["nstep"] = 50

    # Will stop when, twice in a row, the difference
    # between two consecutive evaluations of total energy
    # differ by less than toldfe (in Hartree)
    # This value is way too large for most realistic studies of materials
    # inp['toldfe'] = 1e-8
    inp["tolvrs"] = 1e-10

    return inp

inp_dftp = build_inp_dftp(struct_relaxed)

In [ ]:

def build_flow_dftp(inp, workdir, force=False):

    """Build and return a Flow to compute the dynamical matrix on a (2, 2, 2) qmesh
    as well as DDK and Born effective charges.
    The final DDB with all perturbations will be merged automatically and placed
    in the Flow `outdir` directory.
    """

    flow = flowtk.PhononFlow.from_scf_input(
        workdir, inp, ph_ngqpt=(4, 4, 4), with_becs=False
    )

    if force:
        flow.rmtree()

    return flow

flow_dftp = build_flow_dftp(inp_dftp, "conv_cell/02_dftp")

try:
    scheduler = flow_dftp.make_scheduler()
    scheduler.start()
except:
    pass


# Surfaces

## Surface 100

In [ ]:
def fcc_100_conv(element: str, a: float, n: int, vacuum: float, centered=True):

    h = (n - 1) * a / 2 + vacuum

    coords = np.zeros((2 * n, 3))

    coords[0::4, :] = [[0,   0,   i*a] for i in range((n+1)//2)]
    coords[1::4, :] = [[a/2, a/2, i*a] for i in range((n+1)//2)]
    coords[2::4, :] = [[a/2, 0,   a/2 + i*a] for i in range(n//2)]
    coords[3::4, :] = [[0,   a/2, a/2 + i*a] for i in range(n//2)]
    
    if centered:
        coords[:,-1] += vacuum/2
    
    structure = Structure(
        lattice=Lattice([[a, 0, 0], [0, a, 0], [0, 0, h]]),
        species=2 * n * [element],
        coords=coords,
        coords_are_cartesian=True,
    )
    return structure

In [ ]:
struct_fcc_100 = fcc_100_conv('Al', lattice_constant_relaxed, n=5, vacuum=5*lattice_constant_relaxed/2) 
struct_fcc_100

In [ ]:
# quick_view(struct_fcc_100, supercell=(3,3,2))

fig = Figure()
fig.add_structure(struct_fcc_100, supercell=(3,3,2))
fig.add_unitcell(struct_fcc_100.lattice)
fig.show()

